**Задание:**
1. Используя методы нечёткой логики (например, нечёткое сравнение строк), реализовать алгоритм, выполняющий поиск и маркировку дубликатов в данных, приложенных к заданию.
2. Результатом работы программы должна стать исходная выборка с новой колонкой "Group", в которой должны находиться номера кластеров для каждой записи. Записи, находящиеся в одном кластере, считаются дубликатами друг друга.

Прочитаем датасет:

In [9]:
import pandas as pd
df = pd.read_csv('ign.csv')
df

,score_phrase,title,url,platform,score,genre,release_year
0,Amazing,S.I.M.I.S.,/games/simis/lynx-12176,Lynx,9.000,Action,1999
1,Amazing,S .yI.MbI.S.,/games/simis/ lynx-1217 6,L ynx,9.000,A ntoion,1999
2,Amazing,S.I.M.I.S.,/games/simis/lynx-12176,Lynx,9.000,Action,1999
3,Okay,Namco Museum Vol. 4,/games/namco-museum-vol-4/ps-2089,PlayStation,6.000,"Action, Compilation",1997
4,Okay,Namco Museum Vol. 4,/games/namco-muse um-vol -4/ps-2089,PlayStat ion,6.000,"Action, Compilation",1997
...,...,...,...,...,...,...,...
709,Good,Bomberman Land Touch! 2,/games/bomberman-land-2008/nds-903267,Nintendo DS,7.900,Action,2008
710,Good,Bomberca n Labd Touyh! 2,/g a mes/bomberman-land-2008/nds-9032 67,Ninte nd o DS,7.821,Action,2008
711,Good,Bomberman Land Touch! 2,/games/bomberman -land- 2008/nds-903267,N inte nd o DS,7.821,Action,2008
712,Good,Bomberman Land Touch! 2,/games /bombe rman-land-2008/n ds-903267,Nintendo DS,7.900,Act if n,2008


Выведем типы ячеек:

In [10]:
[type(val) for val in df.iloc[0].values]

[str, str, str, str, numpy.float64, str, numpy.int64]

Считать расстояние между строками будем расстоянием Левенштейна

In [130]:
import numpy as np

def levenshtein_distance(value1, value2):
    rows, cols = len(value1) + 1, len(value2) + 1
    distances = np.zeros((rows, cols))

    for i in range(1, rows):
        for j in range(1, cols):
            distances[i][0] = i
            distances[0][j] = j

    for j in range(1, cols):
        for i in range(1, rows):
            dist = int(value1[i - 1] != value2[j - 1])
            distances[i][j] = min(
                distances[i - 1][j] + 1,
                distances[i][j - 1] + 1,
                distances[i - 1][j - 1] + dist
            )

    total_length = len(value1) + len(value2)
    distance = distances[rows - 1][cols - 1]
    return (total_length - distance) / total_length

Покажем работу функции на пример из датасета:

In [131]:
levenshtein_distance("S.I.M.I.S.", "S .yI.MbI.S.")

0.8636363636363636

В датасете есть три типа столбцов: строки, числа и год

In [132]:
from enum import IntEnum

class ColumnType(IntEnum):
    STRING = 0
    NUMBER = 1
    YEAR = 2

Строки будем сравнивать расстоянием Левенштейна с заданным ratio
Числа будем сравнивать относительно максимального в выборке с заданным ratio
Года будем сравнивать строго

In [158]:
STRING_RATIO = 0.55
NUMBER_RATIO = 0.9

def compare_records(value1, value2, score_maxv):
    score_phrase1, title1, url1, platform1, score1, genre1, release_year1, _ = value1
    score_phrase2, title2, url2, platform2, score2, genre2, release_year2, _ = value2

    score_phrase_ratio = distance(score_phrase1, score_phrase2, ColumnType.STRING)
    title_ratio = distance(title1, title1, ColumnType.STRING)
    url_ratio = distance(url1, url2, ColumnType.STRING)
    platform_ratio = distance(platform1, platform2, ColumnType.STRING)
    score_ratio = distance(score1, score2, ColumnType.NUMBER, maxv=score_maxv)
    genre_ratio = distance(genre1, genre2, ColumnType.STRING)
    release_year_ratio = distance(release_year1, release_year2, ColumnType.YEAR)
    has_equal = (
        score_phrase_ratio > STRING_RATIO and
        title_ratio > STRING_RATIO and
        url_ratio > STRING_RATIO and
        platform_ratio > STRING_RATIO and
        score_ratio > NUMBER_RATIO and
        genre_ratio > STRING_RATIO and
        release_year_ratio == 1
    )
    total_distance = score_phrase_ratio + title_ratio + url_ratio + score_ratio + genre_ratio + release_year_ratio
    return has_equal, total_distance

Добавим столбец с идентификатором кластера

In [159]:
df = df.assign(cluster=0)

Воспользуемся тем, что в данном датасете кластеры идут по возрастанию и будем сравнивать с предыдущей строкой.
В общем случае необходимо сравнивать запись с представителем каждого кластера.

In [160]:
score_maxv = df['score'].max()

for idx in range(1, len(df)):
    is_new_cluster = True
    cur_record = df.values[idx]
    prev_cluster_record = df.values[idx - 1]

    has_equal, _ = compare_records(cur_record, prev_cluster_record, score_maxv)
    df.loc[idx, 'cluster'] = df.loc[idx - 1, 'cluster'] + int(not has_equal)


Выведем полученный столбец идентификатора кластера:

In [163]:
df

,score_phrase,title,url,platform,score,genre,release_year,cluster
0,Amazing,S.I.M.I.S.,/games/simis/lynx-12176,Lynx,9.000,Action,1999,0
1,Amazing,S .yI.MbI.S.,/games/simis/ lynx-1217 6,L ynx,9.000,A ntoion,1999,0
2,Amazing,S.I.M.I.S.,/games/simis/lynx-12176,Lynx,9.000,Action,1999,0
3,Okay,Namco Museum Vol. 4,/games/namco-museum-vol-4/ps-2089,PlayStation,6.000,"Action, Compilation",1997,1
4,Okay,Namco Museum Vol. 4,/games/namco-muse um-vol -4/ps-2089,PlayStat ion,6.000,"Action, Compilation",1997,1
...,...,...,...,...,...,...,...,...
709,Good,Bomberman Land Touch! 2,/games/bomberman-land-2008/nds-903267,Nintendo DS,7.900,Action,2008,226
710,Good,Bomberca n Labd Touyh! 2,/g a mes/bomberman-land-2008/nds-9032 67,Ninte nd o DS,7.821,Action,2008,226
711,Good,Bomberman Land Touch! 2,/games/bomberman -land- 2008/nds-903267,N inte nd o DS,7.821,Action,2008,226
712,Good,Bomberman Land Touch! 2,/games /bombe rman-land-2008/n ds-903267,Nintendo DS,7.900,Act if n,2008,226


Посчитаем accuracy:

In [162]:
sum(
    idx == 0 or
    (df.loc[idx, 'score_phrase'] == df.loc[idx - 1, 'score_phrase']) ==
    (df.loc[idx, 'cluster'] == df.loc[idx - 1, 'cluster'])
    for idx in range(len(df))
) / len(df)

0.9089635854341737